prueba modelo 

In [1]:
import pandas as pd
import boto3
import io
import datetime
from datetime import date, timedelta
from credentials import aws_access_key_id, aws_secret_access_key, region_name
import numpy as np


from gluonts.dataset.pandas import PandasDataset
from gluonts.torch.model.deepar import DeepAREstimator
from gluonts.torch.distributions import NegativeBinomialOutput

In [38]:
def read_s3():
    '''Descarga un archivo desde un bucket de S3 y carga los datos en un DataFrame'''

    s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id,
                      aws_secret_access_key=aws_secret_access_key,
                      region_name=region_name)

    bucket_name = 'proyectotati'
    s3_file_name = 'data.csv'

    try:

        response = s3.get_object(Bucket=bucket_name, Key=s3_file_name)

        # Lee el contenido del archivo en un DataFrame
        df = pd.read_csv(io.BytesIO(response['Body'].read()))
        df['ds'] = pd.to_datetime(df['ds'])

        return df

    except Exception as e:
        print(f'Error al cargar el archivo desde S3: {str(e)}')
        return None



In [44]:
df = read_s3()

df = read_s3()
df.drop_duplicates(inplace=True)

df['y'] = df['y'].astype(int)

df = df.reset_index(drop=True)
df.index

In [33]:
# Especifica la fecha de inicio y fin para el rango de fechas deseado
fecha_fin= pd.to_datetime(date(2023,10,1))
fecha_inicio= pd.to_datetime(date(2019,1,1)) # Reemplaza 'fecha_final' con la fecha de fin adecuada

# Crea un rango de fechas semanal
rango_fechas = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='W')

# Crea un DataFrame con todas las combinaciones posibles de 'ds' y 'unique_id' en el rango de fechas
combinaciones = pd.MultiIndex.from_product([rango_fechas, df['unique_id'].unique()], names=['ds', 'unique_id'])
df_combinado = pd.DataFrame(index=combinaciones).reset_index()
df_combinado.head()

,ds,unique_id
0,2019-01-06,expo_4
1,2019-01-06,expo_5
2,2019-01-06,expo_3
3,2019-01-06,expo_2
4,2019-01-06,impo_3


In [34]:

# Fusiona tu DataFrame original con el DataFrame combinado para llenar los valores faltantes
df2 = df_combinado.merge(df, on=['ds', 'unique_id'], how='left')

# Llena los valores faltantes con 0
df2['y'] = df['y'].fillna(0)

In [45]:
train = df

train_ds = PandasDataset.from_long_dataframe(train,
                                             target='y',
                                             item_id='unique_id',
                                             timestamp='ds',
                                             freq='W')

In [46]:
estimator = DeepAREstimator(freq='W', #la frecuencia de los datos es semanal por lo que el modelo debe ser semanal tambien
                            context_length=10, # el modelo va a usar las ultimas 10 semanas para predecir las siguientes 10, window de 10
                            prediction_length=10, # va a predecir las proximas 10 semanas 
                            num_layers=4, #el modelo posee tres capas con un default de 40 nodos por capa 
                            dropout_rate= 0.2, #seteo de 20% de las units en una layer en cero de forma random
                            trainer_kwargs={ 'max_epochs':10},#210 epochs
                            distr_output= NegativeBinomialOutput()) #el default es Distribucion t-student, pero al usar negative binomial permite que las predicciones no tomen valores negativos

In [47]:
predictor = estimator.train(train_ds)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


c:\Users\Bravo15\anaconda3\envs\modelling-torch\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name  | Type        | Params | In sizes | Out sizes   
----------------------------------------------------------------
0 | model | DeepARModel | 50.0 K | ?        | [1, 100, 10]
----------------------------------------------------------------
50.0 K    Trainable params
0         Non-trainable params
50.0 K    Total params
0.200     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Epoch 0, global step 50: 'train_loss' reached 6.48058 (best 6.48058), saving model to 'c:\\Users\\Bravo15\\Desktop\\raico\\Proyecto final\\Project_Raiconet_101\\Docker Container\\lightning_logs\\version_3\\checkpoints\\epoch=0-step=50.ckpt' as top 1
c:\Users\Bravo15\anaconda3\envs\modelling-torch\lib\site-packages\pytorch_lightning\trainer\call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [41]:
import datetime
from datetime import date

In [42]:
date.today()

datetime.date(2023, 10, 16)

In [48]:
pd.to_datetime(date.today()) - timedelta(days=7)

Timestamp('2023-10-09 00:00:00')